# Decompression

In [20]:
# import
import numpy as np
import cv2 as cv
import math
import os

In [21]:
filename = "BaboonRGB.myjpeg"
filename_without_ext = os.path.splitext(filename)[0]

In [22]:
# Read compressed data from a binary file
import struct
import pickle
import bitarray

def read_binary_file(filename):
    with open(filename, 'rb') as f:
        # Read the height and width
        height, width = struct.unpack('ii', f.read(8))

        # Read the number of Huffman coding tables
        num_tables = struct.unpack('B', f.read(1))[0]

        huffman_tables = []
        encoded_datas = []

        for _ in range(num_tables):
            # Read the length of the coding table
            table_length = struct.unpack('I', f.read(4))[0]

            # Read the serialized coding table and perform deserialization
            serialized_table = f.read(table_length)
            huffman_tables.append(pickle.loads(serialized_table))

            # Read the length of the encoded data in bits
            data_length = struct.unpack('I', f.read(4))[0]
            # Convert the units from bits to bytes
            data_length_in_bytes = math.ceil(data_length/8)

            # Read the encoded data
            data = bitarray.bitarray()
            data.fromfile(f, data_length_in_bytes)
            data = data[:data_length]

            # Convert the bitarray object to a 01 string
            data = data.to01()
            encoded_datas.append(data)

    return huffman_tables, encoded_datas, height, width

huffman_tables, encoded_datas, img_height, img_width = read_binary_file(filename)
dcY_encoding_table, dcCb_encoding_table, dcCr_encoding_table, acY_encoding_table, acCb_encoding_table, acCr_encoding_table = huffman_tables
dcY_encoded_data, dcCb_encoded_data, dcCr_encoded_data, acY_encoded_data, acCb_encoded_data, acCr_encoded_data = encoded_datas

In [23]:
# Perform Entropy Decoding
def huffman_decode(encoding_table, encoded_data):
    decoded_data = []
    reversed_encoding_table = {v: k for k, v in encoding_table.items()}
    current_code = ''

    for bit in encoded_data:
        current_code += bit
        if current_code in reversed_encoding_table:
            char = reversed_encoding_table[current_code]
            decoded_data.append(char)
            current_code = ''

    return decoded_data

dpcmY = np.array(huffman_decode(dcY_encoding_table, dcY_encoded_data))
dpcmCb = np.array(huffman_decode(dcCb_encoding_table, dcCb_encoded_data))
dpcmCr = np.array(huffman_decode(dcCr_encoding_table, dcCr_encoded_data))
rlcY = huffman_decode(acY_encoding_table, acY_encoded_data)
rlcCb = huffman_decode(acCb_encoding_table, acCb_encoded_data)
rlcCr = huffman_decode(acCr_encoding_table, acCr_encoded_data)

In [24]:
# Compute num_block, create zig_blocks(num_block, 64)
num_blockY = dpcmY.shape[0]
num_blockCb = dpcmCb.shape[0]
num_blockCr = dpcmCr.shape[0]
zigY = np.zeros((num_blockY, 64))
zigCb = np.zeros((num_blockCb, 64))
zigCr = np.zeros((num_blockCr, 64))

In [25]:
# DPCM Decoding
def dpcm_decode(dpcm_table, zig_blocks): # (num_block,), (num_block, 64)
    for i in range(zig_blocks.shape[0]):
        if i == 0:
            zig_blocks[i][0] = dpcm_table[i]
        else:
            zig_blocks[i][0] = zig_blocks[i-1][0] + dpcm_table[i]

dpcm_decode(dpcmY, zigY)
dpcm_decode(dpcmCb, zigCb)
dpcm_decode(dpcmCr, zigCr)

In [26]:
# RLC Decoding
def rlc_decode(rlc_table, zig_blocks): # (num_block * ?), (num_block, 64)
    i = 0 # block_index
    j = 1 # block_element_index
    for k in range(len(rlc_table)):
        if rlc_table[k] == (0, 0): # the end of this block
            i = i + 1 # turn to next block
            j = 1
        else: # (skip, value)
            j = j + rlc_table[k][0]
            zig_blocks[i][j] = rlc_table[k][1]
            j = j + 1

rlc_decode(rlcY, zigY)
rlc_decode(rlcCb, zigCb)
rlc_decode(rlcCr, zigCr)

In [27]:
# Inverse Zigzag
def izigzag(linearized_blocks): # (num_block, 64)
    zigzag_table = np.array([[0, 1, 5, 6, 14, 15, 27, 28],
                            [2, 4, 7, 13, 16, 26, 29, 42],
                            [3, 8, 12, 17, 25, 30, 41, 43],
                            [9, 11, 18, 24, 31, 40, 44, 53],
                            [10, 19, 23, 32, 39, 45, 52, 54],
                            [20, 22, 33, 38, 46, 51, 55, 60],
                            [21, 34, 37, 47, 50, 56, 59, 61],
                            [35, 36, 48, 49, 57, 58, 62, 63]])

    blocks = np.zeros((linearized_blocks.shape[0], 8, 8))
    for x in range(blocks.shape[0]):
        for i in range(8):
            for j in range(8):
                blocks[x][i][j] = linearized_blocks[x][zigzag_table[i][j]]

    return blocks # (num_block, 8, 8)

qY = izigzag(zigY)
qCb = izigzag(zigCb)
qCr = izigzag(zigCr)

In [28]:
# Inverse Quantization
Luminance_Quantization_Table = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
                                        [12, 12, 14, 19, 26, 58, 60, 55],
                                        [14, 13, 16, 24, 40, 57, 69, 56],
                                        [14, 17, 22, 29, 51, 87, 80, 62],
                                        [18, 22, 37, 56, 68, 109, 103, 77],
                                        [24, 35, 55, 64, 81, 104, 113, 92],
                                        [49, 64, 78, 87, 103, 121, 120, 101],
                                        [72, 92, 95, 98, 112, 100, 103, 99]])

Chrominance_Quantization_Table = np.array([[17, 18, 24, 47, 99, 99, 99, 99],
                                        [18, 21, 26, 66, 99, 99, 99, 99],
                                        [24, 26, 56, 99, 99, 99, 99, 99],
                                        [47, 66, 99, 99, 99, 99, 99, 99],
                                        [99, 99, 99, 99, 99, 99, 99, 99],
                                        [99, 99, 99, 99, 99, 99, 99, 99],
                                        [99, 99, 99, 99, 99, 99, 99, 99],
                                        [99, 99, 99, 99, 99, 99, 99, 99]])

def iquantization(blocks, quantization_table): # (num_block, 8, 8)
    return blocks * quantization_table # (num_block, 8, 8)

dctY = iquantization(qY, Luminance_Quantization_Table)
dctCb = iquantization(qCb, Chrominance_Quantization_Table)
dctCr = iquantization(qCr, Chrominance_Quantization_Table)

In [29]:
# Perform IDCT on image blocks
def getDctMatrix():
    dct_matrix = np.zeros((8, 8))
    for i in range(8):
        for j in range(8):
            if i == 0:
                dct_matrix[i][j] = 1 / (2*np.sqrt(2))
            else:
                dct_matrix[i][j] = (1.0/2) * np.cos((2*j+1)*i*np.pi/16)
    return dct_matrix

def idct(dct_blocks, dct_matrix): # (num_block, 8, 8)
    blocks = np.zeros_like(dct_blocks)
    for block_index in range(blocks.shape[0]):
        for u in range(8):
            for v in range(8):
                blocks[block_index] = dct_matrix.T @ dct_blocks[block_index] @ dct_matrix

    return blocks # (num_block, 8, 8)

dct_matrix = getDctMatrix()
blocksY = idct(dctY, dct_matrix)
blocksCb = idct(dctCb, dct_matrix)
blocksCr = idct(dctCr, dct_matrix)

In [30]:
# Calculate the size of padded Y, Cb, Cr components based on the size of the original image
def getPaddedSize(h, w): # (heignt, width)
    h = round(h)
    w = round(w)
    h_padding = 0
    w_padding = 0
    if h % 8 != 0:
        h_padding = 8 - (h % 8)
    if w % 8 != 0:
        w_padding = 8 - (w % 8)
    return (h+h_padding, w+w_padding) # (height_padded, width_padded)

imgY_padded = np.zeros(getPaddedSize(img_height, img_width))
imgCb_padded = np.zeros(getPaddedSize(img_height/2, img_width/2))
imgCr_padded = np.zeros(getPaddedSize(img_height/2, img_width/2))

In [31]:
# Merge the 8x8 small blocks into the image
def mergeBlocks(blocks, img): # (num_block, 8, 8), (height_padded, width_padded)
    h, w = img.shape[:2]
    for i in range(h//8):
        for j in range(w//8):
            yoff, xoff = i*8, j*8
            img[yoff:yoff+8, xoff:xoff+8] = blocks[i*(w//8)+j, :, :]

mergeBlocks(blocksY, imgY_padded)
mergeBlocks(blocksCb, imgCb_padded)
mergeBlocks(blocksCr, imgCr_padded)

In [32]:
# Crop the padded image back to the original size
imgY = imgY_padded[:img_height, :img_width]
imgCb = imgCb_padded[:img_height//2, :img_width//2]
imgCr = imgCr_padded[:img_height//2, :img_width//2]

In [33]:
# Perform color upsampling interpolation and merge the three channels
def upSample(y, cb, cr): # (height, width) Y, Cb, Cr
    ycbcr = np.zeros((img_height, img_width, 3))
    for i in range(cb.shape[0]):
        for j in range(cb.shape[1]):
            ycbcr[i*2][j*2][1] = ycbcr[i*2+1][j*2][1] = ycbcr[i*2][j*2+1][1] = ycbcr[i*2+1][j*2+1][1] = cb[i][j]
            ycbcr[i*2][j*2][2] = ycbcr[i*2+1][j*2][2] = ycbcr[i*2][j*2+1][2] = ycbcr[i*2+1][j*2+1][2] = cr[i][j]
    
    ycbcr[:, :, 0] = y
    return ycbcr # (height, width, 3) RGB

imgYCbCr = upSample(imgY, imgCb, imgCr)

In [34]:
# Transform YCbCr to RGB
def ycbcr2rgb(imgYCbCr): # (height, width)
    # Define the transformation matrix
    xform_matrix = np.array([[1, 0, 1.402],
                           [1, -0.344136, -0.714136],
                           [1, 1.772, 0]])
    imgYCbCr[:, :, [1, 2]] -= 128.0
    rgb = np.dot(imgYCbCr, xform_matrix.T)
    return rgb # (height, width)

imgRGB = ycbcr2rgb(imgYCbCr)
imgRGB = np.clip(imgRGB, 0, 255) # https://www.cnblogs.com/sunny-li/p/10265755.html
imgRGB = imgRGB.astype(np.uint8)

In [35]:
# Show and save the decompressed image
imgBGR = cv.cvtColor(imgRGB, cv.COLOR_RGB2BGR)
cv.imshow("result", imgBGR)
cv.waitKey(0)
cv.imwrite(filename_without_ext+"_decompressed"+".bmp", imgBGR)

True